In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Capstone-KPMG/3_Data_Exploration/8_EGrid
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Capstone-KPMG/3_Data_Exploration/8_EGrid
egrid2020_data.xlsx  Generator2021.xlsx  Plant2021.xlsx  Utility2021.xlsx


In [5]:
gen = pd.read_excel('Generator2021.xlsx',header=1)
gen = gen[gen['State']=='WA']
gen['Operation Type'] = 'current'
gen['Operating Year'] = gen['Operating Year'].astype('int')


In [6]:
pro = pd.read_excel('Generator2021.xlsx',sheet_name='Proposed',header=1)
pro = pro[pro['State']=='WA']
pro['Operation Type'] = 'proposed'

In [7]:
total = pd.concat([gen,pro],axis=0)

In [8]:
plant = pd.read_excel('Plant2021.xlsx',header=1)

In [9]:
gen_zip = pd.merge(total,plant.drop(columns=['Utility ID', 'Utility Name']),on='Plant Code',how='left')

In [10]:
import math

In [11]:
gen_zip['Number of Generator']=1

In [12]:
gen_zip['Source Energy'] = gen_zip.apply(lambda x: [x[i] for i in gen_zip.columns if 'Energy Source' in i and 'Planned' not in i and x[i] is not np.NaN and str(x[i])!='nan'],axis=1)

In [13]:
gen_data = gen_zip.groupby(['Latitude','Longitude']).agg({'Nameplate Capacity (MW)':'sum',
                                               'Summer Capacity (MW)':'sum',
                                               'Winter Capacity (MW)':'sum',
                                               'Number of Generator':'size',
                                               'Operating Year':list,
                                               'Source Energy':'sum'}).reset_index()

In [14]:
location_uni = gen_zip[['Latitude','Longitude','Utility Name','Utility ID','Operation Type']].drop_duplicates()
gen_data = pd.merge(location_uni, gen_data,on=['Latitude','Longitude'])

In [15]:
Energy_type = {'ANT': 'Coal',
 'BIT': 'Coal',
 'LIG': 'Coal',
 'SGC': 'Coal',
 'SUB': 'Coal',
 'WC': 'Coal',
 'RC': 'Coal',
 'DFO': 'Petroleum',
 'JF': 'Petroleum',
 'KER': 'Petroleum',
 'PC': 'Petroleum',
 'PG': 'Petroleum',
 'RFO': 'Petroleum',
 'SGP': 'Petroleum',
 'WO': 'Petroleum',
 'BFG': 'Natural Gas/Other Gas',
 'NG': 'Natural Gas/Other Gas',
 'H2': 'Natural Gas/Other Gas',
 'OG': 'Natural Gas/Other Gas',
 'AB': 'Solid Renewable Fuels',
 'MSW': 'Solid Renewable Fuels',
 'OBS': 'Solid Renewable Fuels',
 'WDS': 'Solid Renewable Fuels',
 'OBL': 'Liquid Renewable Fuels',
 'SLW': 'Liquid Renewable Fuels',
 'BLQ': 'Liquid Renewable Fuels',
 'WDL': 'Liquid Renewable Fuels',
 'LFG': 'Gas Renewable Fuels',
 'OBG':'Gas Renewable Fuels',
 'SUN': 'Other Renewable Fuels',
 'WND': 'Other Renewable Fuels',
 'GEO': 'Other Renewable Fuels',
 'WAT': 'Other Renewable Fuels',
 'WAT': 'Other',
 'NUC': 'Other',
 'PUR': 'Other',
 'WH': 'Other',
 'TDF': 'Other',
 'MWH': 'Other',
 'OTH': 'Other'}

In [16]:
gen_data.head()

,Latitude,Longitude,Utility Name,Utility ID,Operation Type,Nameplate Capacity (MW),Summer Capacity (MW),Winter Capacity (MW),Number of Generator,Operating Year,Source Energy
0,47.079722,-122.365,Puget Sound Energy Inc,15500,current,177.8,134.0,162.2,2,"[1981.0, 1981.0]","[NG, DFO, NG, DFO]"
1,46.4711,-119.3339,Energy Northwest,20160,current,1200.0,1151.0,1172,1,[1984.0],[NUC]
2,48.620336,-118.111059,Avista Corp,20169,current,57.9,56.2,58.7,2,"[1983.0, 2002.0]","[WDS, NG, NG]"
3,48.4558,-122.4358,Puget Sound Energy Inc,15500,current,376.0,280.0,348,4,"[1984.0, 1984.0, 2001.0, 2001.0]","[NG, DFO, NG, DFO, NG, DFO, NG, DFO]"
4,47.699167,-121.783333,City of Seattle - (WA),16868,current,16.8,16.8,16.8,1,[1995.0],[WAT]


In [17]:
gen_data['Energy Type'] = gen_data['Source Energy'].apply(lambda x: pd.Series([Energy_type[i] for i in x]).unique())

In [18]:
gen_data['Color'] = gen_data['Energy Type'].apply(lambda x: 'green' if 'Renewable' in str(x) else 'red')
gen_data['Shape'] = gen_data['Operation Type'].apply(lambda x: 'circle' if x == 'proposed' else 'circle-dot')

In [19]:
gen_data['Renewable/Nonrenewable'] = gen_data['Color'].apply(lambda x: 'Renewable' if x == 'green' else 'Nonrenewable')

In [20]:
gen_data[gen_data['Operation Type']=='current'].groupby('Renewable/Nonrenewable').agg({'Utility ID':'size',
                                                                                       'Number of Generator':'sum',
                                                                                       'Nameplate Capacity (MW)':'sum',
                                                                                       }).rename(columns={'Utility ID':'Number of Plants'})

,Number of Plants,Number of Generator,Nameplate Capacity (MW)
Renewable/Nonrenewable,,,
Nonrenewable,98,351,27711.9
Renewable,39,72,3755.4


In [21]:
4239.0/(27731.9+4239.0)

0.1325893234159814

In [22]:
gen_data.groupby('Renewable/Nonrenewable').agg({'Utility ID':'size',
                                                                                       'Number of Generator':'sum',
                                                                                       'Nameplate Capacity (MW)':'sum',
                                                                                       }).rename(columns={'Utility ID':'Number of Plants'})

,Number of Plants,Number of Generator,Nameplate Capacity (MW)
Renewable/Nonrenewable,,,
Nonrenewable,99,359,27731.9
Renewable,46,80,4239.0


In [23]:
gen_data.groupby(['Operation Type','Renewable/Nonrenewable']).agg({'Utility ID':'size',
                                                                   'Number of Generator':'sum',
                                                                   'Nameplate Capacity (MW)':'sum'
                                                                   }).rename(columns={'Utility ID':'Number of Plants'})

Number of Plants  Number of Generator  \
Operation Type Renewable/Nonrenewable                                          
current        Nonrenewable                          98                  351   
               Renewable                             39                   72   
proposed       Nonrenewable                           1                    8   
               Renewable                              7                    8   

                                       Nameplate Capacity (MW)  
Operation Type Renewable/Nonrenewable                           
current        Nonrenewable                            27711.9  
               Renewable                                3755.4  
proposed       Nonrenewable                               20.0  
               Renewable                                 483.6

In [24]:
import folium
from folium.plugins import BeautifyIcon

import numpy as np

# grab the center of our map using our cool function
center = 47.7511, -120.7401

# Initialize Folium Map again (same as before)
m = folium.Map(location=center, 
               zoom_start=7,
               width=800,height=600)


# Location of EV chargers  
for i in range(gen_data.shape[0]):
   folium.Marker(
       location = [gen_data.iloc[i]['Latitude'], gen_data.iloc[i]['Longitude']],
       #icon=folium.Icon(color=gen_data.iloc[i]['Color']),
       icon = BeautifyIcon(border_color = gen_data.iloc[i]['Color'],
                           border_width=5,
                           icon_shape = gen_data.iloc[i]['Shape']),
       popup = str(gen_data.iloc[i]['Utility Name'])+'\nNameplate Capacity: '+ 
       str(gen_data.iloc[i]['Nameplate Capacity (MW)'])+'\nSummer Capacity: '+
       str(gen_data.iloc[i]['Summer Capacity (MW)'])+'\nWinter Capacity: '+
       str(gen_data.iloc[i]['Winter Capacity (MW)'])+'\nNumber of Generator: '+
       str(gen_data.iloc[i]['Number of Generator'])+'\nOperating Year: '+
       str(gen_data.iloc[i]['Operating Year'])+'\nEnergy Type: '+
       str(gen_data.iloc[i]['Energy Type'])
   ).add_to(m)

m

In [29]:
%cd /content/drive/My Drive/Capstone-KPMG/3_Data_Exploration/3_EVChargingStationLocations
!ls


/content/drive/My Drive/Capstone-KPMG/3_Data_Exploration/3_EVChargingStationLocations
'alt_fuel_stations(Sep_25_2022).csv'


In [30]:
EV_station = pd.read_csv('alt_fuel_stations(Sep_25_2022).csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (20,31,39,46) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [31]:
EV_station = EV_station[EV_station['Fuel Type Code']=='ELEC']
EV_station = EV_station[EV_station['State']=='WA']

In [33]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Capstone-KPMG/3_Data_Exploration/12_Gas_Station
!ls
fuel = pd.read_csv('fuel_point.csv').rename(columns={'Y':'Latitude','X':'Longitude'})

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Capstone-KPMG/3_Data_Exploration/12_Gas_Station
fuel_point.csv	fuel_polygon.csv


In [34]:
fuel_poly = pd.read_csv('fuel_polygon.csv').rename(columns={'Y':'Latitude','X':'Longitude'})

In [35]:
fuel.columns

Index(['Longitude', 'Latitude', 'amenity', 'name', 'brand', 'addr:housenumber',
       'addr:street', 'addr:postcode', 'addr:city', 'phone', 'opening_hours',
       'website', 'operator', 'fuel:diesel', 'addr:state', 'name:en',
       'brand:wikidata', 'brand:wikipedia', 'addr:country', 'building', 'shop',
       'fuel:octane_87', 'fuel:biodiesel', 'access', 'fuel:e10',
       'fuel:gasoline', 'fuel:octane_92', 'is_in', 'atm', 'car_wash',
       'fuel:octane_91', 'membership', 'fuel:lpg', 'fuel:octane_89',
       'fuel:octane_95', 'wheelchair'],
      dtype='object')

In [36]:
fuel_poly.columns

Index(['Longitude', 'Latitude', 'amenity', 'building', 'name', 'addr:street',
       'addr:housenumber', 'brand', 'addr:postcode', 'addr:city', 'website',
       'operator', 'fuel:diesel', 'addr:state', 'brand:wikidata',
       'brand:wikipedia', 'opening_hours', 'designation', 'phone', 'shop',
       'fuel:octane_87', 'fuel:octane_92', 'layer', 'addr:country',
       'fuel:octane_89', 'fuel:octane_91', 'is_in', 'building:levels',
       'access', 'name:en', 'membership', 'fuel:e10', 'fuel:biodiesel'],
      dtype='object')

In [37]:
fuel

,Longitude,Latitude,amenity,name,brand,addr:housenumber,addr:street,addr:postcode,addr:city,phone,...,fuel:octane_92,is_in,atm,car_wash,fuel:octane_91,membership,fuel:lpg,fuel:octane_89,fuel:octane_95,wheelchair
0,-122.329806,48.393760,fuel,24 Hour Express Fuel,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-120.916225,47.191712,fuel,76,76,1001.0,East 1st Street,98922,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-122.357577,47.238704,fuel,76,76,2002.0,54th Avenue E,98424-1904,Fife,NaN,...,NaN,Fife; Washington; USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-122.291324,48.435227,fuel,76,76,NaN,NaN,NaN,NaN,NaN,...,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-120.014523,47.839608,fuel,76,76,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,-122.486406,48.855645,fuel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
884,-122.486787,48.856022,fuel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
885,-122.265343,48.994026,fuel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
886,-123.066954,48.984814,fuel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
EV_station = EV_station[['Latitude','Longitude']]
EV_station['type'] = 'EV'
EV_station['color'] = 'green'

<ipython-input-38-24a886f88f6e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EV_station['type'] = 'EV'
<ipython-input-38-24a886f88f6e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EV_station['color'] = 'green'


In [39]:
fuel = fuel[['Latitude','Longitude']]
fuel['type'] = 'gas'
fuel['color'] = 'red'

<ipython-input-39-f008c418bd4b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuel['type'] = 'gas'
<ipython-input-39-f008c418bd4b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuel['color'] = 'red'


In [40]:
fuel_poly = fuel_poly[['Latitude','Longitude']]
fuel_poly['type'] = 'gas'
fuel_poly['color'] = 'red'

<ipython-input-40-9f7192ace0f0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuel_poly['type'] = 'gas'
<ipython-input-40-9f7192ace0f0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fuel_poly['color'] = 'red'


In [41]:
stations = pd.concat([fuel,fuel_poly,EV_station],axis=0)

In [42]:
import folium

import numpy as np

# grab the center of our map using our cool function
center = 47.7511, -120.7401

# Initialize Folium Map again (same as before)
m = folium.Map(location=center, 
               zoom_start=7,
               width=800,height=600)


# Location of EV chargers  
for i in range(stations.shape[0]):
   folium.Circle(
       location = [stations.iloc[i]['Latitude'], stations.iloc[i]['Longitude']],
        radius=30,
        fill=True,
        color=stations.iloc[i]['color'],
        fill_opacity=0.2,
        opacity=0.5
   ).add_to(m)

m

In [43]:
fuel

,Latitude,Longitude,type,color
0,48.393760,-122.329806,gas,red
1,47.191712,-120.916225,gas,red
2,47.238704,-122.357577,gas,red
3,48.435227,-122.291324,gas,red
4,47.839608,-120.014523,gas,red
...,...,...,...,...
883,48.855645,-122.486406,gas,red
884,48.856022,-122.486787,gas,red
885,48.994026,-122.265343,gas,red
886,48.984814,-123.066954,gas,red
